# CSE 258 Assignment 1 Rating
**Ming Ki Toby Cheng**

In [39]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
from sklearn import linear_model
import numpy
import random
import matplotlib.pyplot as plt

In [40]:
def readGz(path):
    for l in gzip.open(path, "rt"):
        yield eval(l)


def readCSV(path):
    f = gzip.open(path, "rt")
    f.readline()
    for l in f:
        yield l.strip().split(",")

In [41]:
# Defining MSE function
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [42]:
def prediction(user, item):
    if user not in userBiases and item in itemBiases:
        return alpha + itemBiases[item]
    if user in userBiases and item not in itemBiases:
        return alpha + userBiases[user]
    if user not in userBiases and item not in itemBiases:
        return alpha
    if user in userBiases and item in itemBiases:
        return alpha + userBiases[user] + itemBiases[item]

In [43]:
users = []
books = []
ratings = []

for user, book, _ in readCSV("train_Interactions.csv.gz"):
    users.append(user)
    books.append(book)
    ratings.append(_)

In [44]:
# Re-initializing data
users_train = users[:190000]
books_train = books[:190000]
ratings_train = ratings[:190000]
users_valid = users[190000:]
books_valid = books[190000:]
ratings_valid = ratings[190000:]

In [45]:
training_rating = list(zip(users_train, books_train, ratings_train))
validation_rating = list(zip(users_valid, books_valid, ratings_valid))

In [46]:
train_rating = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)


for user, book, r in training_rating:
    train_rating.append(int(r))
    userRatings[user].append(int(r))
    bookRatings[book].append(int(r))


In [47]:
trainAverage = sum(train_rating) / len(train_rating)

alpha = trainAverage
nUsers = len(userRatings)
nItems = len(bookRatings)
users = list(userRatings.keys())
items = list(bookRatings.keys())

userBiases = defaultdict(float)
itemBiases = defaultdict(float)


In [48]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [49]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in training_rating]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [50]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(training_rating)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in training_rating:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - int(d[2])
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

In [51]:
labels = [int(d[2]) for d in training_rating]

In [107]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 0.0000168))

MSE = 1.4806916922249362
MSE = 2.300129305099145
MSE = 1.4733843875018926
MSE = 1.4732252256322795
MSE = 1.472589045174189
MSE = 1.4700517956676158
MSE = 1.4600223548536508
MSE = 1.4218175069968622
MSE = 1.070216311137085
MSE = 1.1392270770979693
MSE = 1.015447885961261
MSE = 0.9931982668371578
MSE = 0.9543919633735973
MSE = 0.942082305551534
MSE = 0.9362790208580444
MSE = 0.9324379184063721
MSE = 0.9283485644493698
MSE = 0.9237191530780602
MSE = 0.9223042482718358
MSE = 0.9224755182136947
MSE = 0.9208647934180683
MSE = 1.0443647242062977
MSE = 0.9208597995802826
MSE = 0.9208640557390156
MSE = 0.9209496756164541
MSE = 0.9208986314865779
MSE = 0.9207212412243876
MSE = 0.9203456334394361
MSE = 0.920493383192212
MSE = 0.9203020576035599
MSE = 0.9202313593298367
MSE = 0.9201388579393541
MSE = 0.9200958606191081
MSE = 0.920077018929819
MSE = 0.9200851697200259
MSE = 0.9200782918069236
MSE = 0.9200795786163819
MSE = 0.9200728073906611
MSE = 0.920069884545226
MSE = 0.9200687266953449
MSE = 0.

(array([ 3.81259351, -0.00758701,  0.24722628, ..., -0.01909725,
        -0.29646326, -0.6235612 ]),
 0.9930888871038479,
 {'grad': array([-1.27239206e-04, -4.89260821e-08, -4.57111230e-08, ...,
          1.52263815e-07,  2.28627730e-07,  2.67641548e-07]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 79,
  'nit': 62,
  'warnflag': 0})

In [108]:
val_pred = []
real_rate = []
for user, book, rating in validation_rating:
    val_pred.append(prediction(user,book))
    real_rate.append(int(rating))

In [109]:
MSE(val_pred,real_rate)

1.1080579217390374

0.0000175        0.0000169          0.0000168          0.0000167
1.10807495437515 1.1080549096625298 1.1080412812036005 1.1080477739367218

0.000014           0.000016          0.000017           0.000018
1.1083714213430038 1.108064308947268 1.1080559417223608 1.108135090292363

0.000009          0.0000095          0.000015           0.00002           0.000012
1.111127818180121 1.1106934089828682 1.1081628940572545 1.108512995969658 1.1090584867923212

lambda:0.0001,  0.00001             0.000005            0.000008          0.0000095
MSE: 1.17724    1.1103061733208988  1.1163570457783654  1.1121307718734341

lambda: 0.001, 1, 3
MSE: 1.39810, 1.4907, 1.4908

In [61]:
# Writing predictions of test set to file
predictions = open("predictions_Rating_Assignment1_v3.0.txt", "w")
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    
    predictions.write(u + "-" + b + "," + str(prediction(u,b)) + "\n")

predictions.close()

#### Using previous lambda now use all the data available

In [ ]:
all_ratngs = list(zip(users, books, ratings))

In [ ]:
# Forming dictionary with user and book combinations
ratingPerCombo = {}
usersPerBook = defaultdict(set)
booksPerUser = defaultdict(set)

for user, book,r in all_ratings:
    usersPerBook[book].add(user)
    booksPerUser[user].add(book)
    ratingPerCombo[(user,book)] = int(r)

In [ ]:
# Initializing all user ratings and all books dictionaries
totalRatings = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)

# All ratings for each user and each book
for user, book, r in all_ratings:
    r = int(r)
    totalRatings.append(r)
    userRatings[user].append(r)
    bookRatings[book].append(r)

globalAverage = sum(totalRatings) / len(totalRatings)
userAverage = {}
userBias = {}
user_total = 0
bookAverage = {}
bookBias = {}
book_total = 0

# Initializing user and book biases
for user in userRatings:
    userAverage[user] = sum(userRatings[user]) / len(userRatings[user])
    
for user in userAverage:
    user_total += float(userAverage[user])
    
for user in userAverage:
    userBias[user] = userAverage[user] - (user_total/len(userAverage))

for book in bookRatings:
    bookAverage[book] = sum(bookRatings[book]) / len(bookRatings[book])
    
for book in bookAverage:
    book_total += float(bookAverage[book])
    
for book in bookAverage:
    bookBias[book] = bookAverage[book] - (book_total/len(bookAverage))

In [ ]:
# Defining alpha, bookBias, userBias by convergence from Training data and lambda value 1
lamb = 3.15
alpha_sum = 0
alpha = 0
MSE_diff = 5
trial = 0

while MSE_diff > 0.00005 or trial > 1000:
    model_predictions = []
    actual_rating = []
    alpha_sum = 0
    
    for user, book, r in all_ratings:
        alpha_sum += (r-(userBias[user] + bookBias[book]))
    alpha = alpha_sum/len(all_ratings)
    
    for user in booksPerUser:
        beta_U_Sum = 0
        for books in booksPerUser[user]:
            beta_U_Sum += ratingPerCombo[user,books] - (alpha + bookBias[books])
        userBias[user] = beta_U_Sum/(lamb+ len(booksPerUser[user]))
    
    for book in usersPerBook:
        beta_I_Sum = 0
        for users in usersPerBook[book]:
            beta_I_Sum += ratingPerCombo[users,book] - (alpha + userBias[users])
        bookBias[book] = beta_I_Sum/(lamb+ len(usersPerBook[book]))
    
    for user, book, r in all_ratings:
        model_predictions.append(int(prediction(user, book)))
        actual_rating.append(r)
    
    if trial == 0:
        MSE_0 = 0

    MSE_1 = MSE(model_predictions, actual_rating)
    
    MSE_diff = abs(MSE_1 - MSE_0)

    print('MSE_new:', MSE_1)
    
    MSE_0 = MSE_1
    trial += 1 

In [ ]:
# Predictions and MSE on validation data
model_predictions = []
for user, book, r in validation_rating:
    model_predictions.append(prediction(user, book))

MSE_valid = MSE(model_predictions, ratings_valid)
print('MSE on validation set:',MSE_valid)

In [ ]:
# Writing predictions of test set to file
predictions = open("predictions_Rating_Assignment1.txt", "w")
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    
    predictions.write(u + "-" + b + "," + str(prediction(u,b)) + "\n")

predictions.close()

**Kaggle Username: tobycheng or Toby Cheng**

**Kaggle Rating MSE: 1.13707**